In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np
import pandas as pd

URL_BASE = "https://www.mwcbarcelona.com"

Obtener pagina principal, acpetar cookie

In [2]:
def call_driver(url):
    driver = webdriver.Chrome()
    driver.get(url + '/exhibitors')
    driver.implicitly_wait(0.5)
    driver.maximize_window()
    return driver

def cookie_handler(driver):
    button = driver.find_element(By.CLASS_NAME, value = 'banner-actions-container').click()

Obtener la info de cada participante: nombre, descripcio, redes sociales etc

In [3]:
def getting_info(driver):

    name = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1'))).text
    text = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'p'))).text
    
    class_loc = "px-4 py-1 mb-2 mr-2 text-xs no-underline bg-gray-100 border border-gray-300 rounded-full"
    xpath = f"//p[@class='{class_loc}']"
    location = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, xpath))).text
    
    class_tags = "px-4 py-1 mb-2 mr-2 text-xs no-underline bg-gray-100 border border-gray-300 rounded-full"
    xpath = f"//li[@class='{class_tags}']"
    try:
        tags_list = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
        array_tags = [[tags.text for tags in tags_list]]

    except:
        array_tags = [[None]]
    
    social_path = "//div[contains(@class, 'sm:flex flex-row mt-2')]//a[@target='_blank']"
    
    try:
        social_links = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, social_path)))
        social_array = [[s.get_attribute("href") for s in social_links]]
    
    except: 
        social_array = [[None]]
    
    web_path = "//li[contains(@class, 'flex w-full md:w-1/3 pr-2')]//a[@target='_blank']"
    try:
        web_page = (WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, web_path))).get_attribute('href'))
    except:
        web_page = None
        
    df = pd.DataFrame(columns=['Name', 'Text', 'Location', 'Tags', 'Social_media', 'Web'])
    df['Name'] = [name]
    df['Text'] = [text]
    df['Location'] = [location]
    df['Tags'] = array_tags
    df['Social_media']  = social_array
    df['Web'] = web_page
    
    return df

Crear dataframe por cada subpagina

In [4]:
def create_df(list_df, list_links):
    links_df = pd.DataFrame(list_links)
    result_df = pd.concat(list_df, ignore_index=True)
    result_df['Links'] = links_df
    
    return result_df

Raspar cada subpagina y devuelve un df completo

In [5]:
def it_pag(driver):
    element_tr = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, '//tr[@role="link"]')))
    links_list = [URL_BASE + i.get_attribute("data-href") for i in element_tr]

    df_list = []
    for i in links_list:
        driver.execute_script("window.open("");") 
        driver.switch_to.window(driver.window_handles[1])  
        try:
            driver.get(i)
            driver.implicitly_wait(10)
            df_list.append(getting_info(driver))
            driver.close()

            driver.switch_to.window(driver.window_handles[0]) 

        except:
            print(f'Error al procesar la página {i}')
            continue
    df = create_df(df_list, links_list)
    return df

Funcion para obtener los links de cada subpag

In [8]:
def subb_get(driver):
    subpages_path = "//li[contains(@class, 'ais-Pagination-item ais-Pagination-item--page')]//a[@class='ais-Pagination-link']"
    subpages_links = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.XPATH, subpages_path)))
    return subpages_links

In [9]:
driver = call_driver(URL_BASE)
cookie_handler(driver)
driver.implicitly_wait(10)

subpages_links = subb_get(driver)

while len(subpages_links) == 1:
    subpages_links = subb_get(driver)
    time.sleep(5)
    
subpages_arrays = [sub.get_attribute("href") for sub in subpages_links ]
print('Subpages:')
subpages_arrays

Subpages:


['https://www.mwcbarcelona.com/exhibitors',
 'https://www.mwcbarcelona.com/exhibitors?page=2',
 'https://www.mwcbarcelona.com/exhibitors?page=3',
 'https://www.mwcbarcelona.com/exhibitors?page=4',
 'https://www.mwcbarcelona.com/exhibitors?page=5']

Raspar cada subpagina y entrar a cada conferecista y obtener su informacion, cada pagina es devuelta como un df el cual es guardado en una lista, la cual es conatenada al final

In [10]:
df_list = []
for i in subpages_arrays:
    try:
        driver.get(i)
        driver.implicitly_wait(15)
        df_per_p = it_pag(driver)
        df_list.append(df_per_p)
        
        driver.switch_to.window(driver.window_handles[0]) 
        
    except:
        print(f'Error al procesar la página {i}')
        continue
driver.quit()        
result_df = pd.concat(df_list, ignore_index=True).set_index('Name')
result_df

Error al procesar la página https://www.mwcbarcelona.com/exhibitors/26674-accenture-llc
Error al procesar la página https://www.mwcbarcelona.com/exhibitors?page=4
Error al procesar la página https://www.mwcbarcelona.com/exhibitors?page=5


,Text,Location,Tags,Social_media,Web,Links
Name,,,,,,
Orange,Orange is one of the world’s leading telecommu...,Hall 3 Stand 3K10,"[ARTIFICIAL INTELLIGENCE, 5G/6G, CYBERSECURITY...",[https://x.com/intent/tweet?text=I%20am%20at%2...,https://www.orange.com/en,https://www.mwcbarcelona.com/exhibitors/26349-...
Ribbon,Ribbon Communications (Nasdaq: RBBN) delivers ...,Hall 2 Stand 2C42,"[ARTIFICIAL INTELLIGENCE, CLOUD SERVICES, 5G/6...",[https://x.com/intent/tweet?text=I%20am%20at%2...,https://www.rbbn.com/,https://www.mwcbarcelona.com/exhibitors/26348-...
Tecnotree Corporation,Tecnotree Corporation: more company informatio...,Hall 5 Stand 5F81,[None],[https://x.com/intent/tweet?text=I%20am%20at%2...,None,https://www.mwcbarcelona.com/exhibitors/26778-...
+Switzerland Global Enterprise,We are the organization of Swiss Pavilions at ...,Hall 5 Stand 5C30,"[ARTIFICIAL INTELLIGENCE, 5G/6G, CYBERSECURITY...",[https://x.com/intent/tweet?text=I%20am%20at%2...,https://www.s-ge.com/,https://www.mwcbarcelona.com/exhibitors/26637-...
1nce GmbH,1nce GmbH: more company information coming soon.,Hall 3 Stand OA3B.20,[None],[https://x.com/intent/tweet?text=I%20am%20at%2...,None,https://www.mwcbarcelona.com/exhibitors/27268-...
...,...,...,...,...,...,...
AL YAH SATELLITE COMPANY PJRC,AL YAH SATELLITE COMPANY PJRC: more company in...,Hall 7 Stand 7A9Ex,[None],[https://x.com/intent/tweet?text=I%20am%20at%2...,None,https://www.mwcbarcelona.com/exhibitors/27239-...
Alarislabs Pte Ltd,Alarislabs Pte Ltd: more company information c...,Hall Congress Square Stand CS96,[None],[https://x.com/intent/tweet?text=I%20am%20at%2...,None,https://www.mwcbarcelona.com/exhibitors/26379-...
Alea Soluciones SL,Alea Soluciones SL: more company information c...,Hall 4 Stand 4C30,[None],[https://x.com/intent/tweet?text=I%20am%20at%2...,None,https://www.mwcbarcelona.com/exhibitors/27772-...


In [13]:
result_df.to_csv('MWCBarcelona.csv')